# Exercice for fun - Create an Chatbot 

This chatbot should have a short term summarized memory and a long term memory about user infos.
The short term memory should trim the history messages to last 4 messages.

In [1]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, trim_messages


In [2]:
MODEL = "cogito:8b"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"
ctx_factor = 1
pred_factor = 1

In [3]:
chat_model = ChatOllama(model=MODEL, temperature=0.8, num_gpu=256, num_ctx=4092 * ctx_factor, num_predict=1000 * pred_factor)

In [4]:
workflow = StateGraph(state_schema=MessagesState)

In [5]:
trimmer = trim_messages(strategy="last", max_tokens=3, token_counter=len)

In [6]:
# Define the call function
def call_model(state: MessagesState):
    system_prompt = (
        # DEEP_THINKING_INSTRUCTION +
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability. "
        "The provided chat history includes a summary of the earlier conversation."
    )
    system_message = SystemMessage(content=system_prompt)

    # trimmed_messages = trimmer.invoke(state["messages"])

    print("State messages", len(state["messages"]),"\n", state["messages"])
    print()
    message_history = state["messages"][:-3]
    kept_messages = state["messages"][-3:]

    if len(message_history) >= 4:
        summary_prompt = (
            # DEEP_THINKING_INSTRUCTION +
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        summary_message = chat_model.invoke(
            message_history + [HumanMessage(content=summary_prompt)]
        )

        delete_messages = [RemoveMessage(id=msg.id) for msg in message_history]
        print("Delete", len(delete_messages), "\n", delete_messages)
        print()

        # human_message = HumanMessage(content=last_human_message.content)
        print("System msg\n", system_message)
        print()
        print("Summary msg\n", summary_message)
        print()
        # print("Human msg\n", trimmed_messages)
        print()
        # print("Raw msg\n", *raw_messages_to_keep)
        # messages = [system_message, summary_message] + trimmed_messages
        messages = [system_message, summary_message] + kept_messages
        print("Messages\n", messages)
        print()
        # response = chat_model.invoke([system_message, summary_message, *raw_messages_to_keep, human_message])
        response = chat_model.invoke(messages)
        # print("Response", response)
        # print()

        # messages_updates = [summary_message, human_message, response] + delete_messages
        # messages_updates = [summary_message] + trimmed_messages + [response] + delete_messages
        messages_updates = [summary_message] + kept_messages + [response] + delete_messages
        # messages_updates = "END"
    else:
        messages_updates = chat_model.invoke([system_message] + state["messages"])
    print("MSG Update", *messages_updates)
    print()
    return {"messages": messages_updates}


In [7]:
# Define the node and edge
workflow.add_node("chatbot", call_model)
workflow.add_edge(START, "chatbot")

In [8]:
checkpointer = InMemorySaver()
chatbot = workflow.compile(checkpointer=checkpointer)

In [9]:
config = {"configurable": {"thread_id": "1"}}

In [10]:
# # Loop of discussion

# for chunk in chatbot.stream(
#     input={"messages": [{"role": "user", "content": input()}]},
#     config=config,
#     stream_mode="values"
# ):
#     # print(chunk)  #["messages"][-1])  #.pretty_print()
#     # print(chunk["messages"][-1])  #.pretty_print()
#     # print()
#     chunk["messages"][-1].pretty_print()

In [11]:
# Loop of discussion
stop_tag = False

from time import sleep

while True:
    break_point = False
    for i, chunk in enumerate(chatbot.stream(
        input={"messages": [{"role": "user", "content": input()}]},
        config=config,
        stream_mode="values"
    )):
        print(i, stop_tag)
        chunk["messages"][-1].pretty_print()
        # print(chunk["messages"][-1].content)
        sleep(1)
        if "bye" in chunk["messages"][-1].content.lower():
            stop_tag = True
        if stop_tag and i == 1:
            break_point = True
    if break_point:
        break

0 False
================================ Human Message =================================

Bonjour
State messages 1 
 [HumanMessage(content='Bonjour', additional_kwargs={}, response_metadata={}, id='1c48b252-6907-4ff9-bf25-6385bb7952b3')]

MSG Update ('content', "Bonjour ! Comment puis-je vous aider aujourd'hui ?") ('additional_kwargs', {}) ('response_metadata', {'model': 'cogito:8b', 'created_at': '2025-06-30T03:15:50.2562004Z', 'done': True, 'done_reason': 'stop', 'total_duration': 575918000, 'load_duration': 20887000, 'prompt_eval_count': 44, 'prompt_eval_duration': 127652800, 'eval_count': 12, 'eval_duration': 425282400, 'model_name': 'cogito:8b'}) ('type', 'ai') ('name', None) ('id', 'run-83a07a44-e6f5-49d5-ad90-b3e1323f8dd7-0') ('example', False) ('tool_calls', []) ('invalid_tool_calls', []) ('usage_metadata', {'input_tokens': 44, 'output_tokens': 12, 'total_tokens': 56})

1 False
================================== Ai Message ==================================

Bonjour ! Comment p

#### Next: Working base

In [12]:
# # Loop of discussion
# stop_tag = False

# from time import sleep

# while True:
#     human_msg = [HumanMessage(input())]
#     output = chatbot.invoke(input={"role": "user", "messages": human_msg}, config=config)
#     output["messages"][-1].pretty_print()
#     print()

#     # if "bye" in output["messages"][-1].content.lower():
#     #     stop_tag = True
#     # if stop_tag and i == 1:
#     #     break